## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-18-21-58-00 +0000,world,10 Americans Released From Venezuela in Exchan...,https://www.wsj.com/world/americas/10-american...
1,2025-07-18-21-57-00 +0000,world,Opinion | Europe Gets Tougher on Putin,https://www.wsj.com/opinion/europe-sanctions-v...
2,2025-07-18-21-44-00 +0000,world,Opinion | Wall Street Misprices South Africa’s...,https://www.wsj.com/opinion/wall-street-mispri...
3,2025-07-18-19-55-00 +0000,world,U.S. Reshuffles Arms Deliveries to Free Up Mor...,https://www.wsj.com/politics/national-security...
4,2025-07-18-19-03-00 +0000,world,Brazil’s Bolsonaro Ordered to Wear Ankle Bracelet,https://www.wsj.com/world/americas/brazils-bol...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,europe,2
54,china,2
0,americans,1
3,exchange,1
1,released,1


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-07-18-03-00-00 +0000,world,German Startup Wants to Regrow Europe’s ‘Spine...,https://www.wsj.com/world/europe/german-startu...,11
8,2025-07-18-09-38-00 +0000,world,U.S.-Based Wells Fargo Banker Blocked From Lea...,https://www.wsj.com/world/china/wells-fargo-ba...,8
11,2025-07-18-03-00-00 +0000,world,China’s New Plan to Encourage More Births: $50...,https://www.wsj.com/world/china/chinas-new-pla...,7
3,2025-07-18-19-55-00 +0000,world,U.S. Reshuffles Arms Deliveries to Free Up Mor...,https://www.wsj.com/politics/national-security...,7
0,2025-07-18-21-58-00 +0000,world,10 Americans Released From Venezuela in Exchan...,https://www.wsj.com/world/americas/10-american...,7


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,11,2025-07-18-03-00-00 +0000,world,German Startup Wants to Regrow Europe’s ‘Spine...,https://www.wsj.com/world/europe/german-startu...
8,8,2025-07-18-09-38-00 +0000,world,U.S.-Based Wells Fargo Banker Blocked From Lea...,https://www.wsj.com/world/china/wells-fargo-ba...
0,7,2025-07-18-21-58-00 +0000,world,10 Americans Released From Venezuela in Exchan...,https://www.wsj.com/world/americas/10-american...
7,7,2025-07-18-13-39-00 +0000,world,U.K. Names Russian Spies and Denounces Moscow’...,https://www.wsj.com/world/europe/u-k-names-rus...
3,7,2025-07-18-19-55-00 +0000,world,U.S. Reshuffles Arms Deliveries to Free Up Mor...,https://www.wsj.com/politics/national-security...
12,7,2025-07-18-01-00-00 +0000,world,Israel’s Red Lines in Syria Are Drawing It Dee...,https://www.wsj.com/world/middle-east/israel-d...
2,6,2025-07-18-21-44-00 +0000,world,Opinion | Wall Street Misprices South Africa’s...,https://www.wsj.com/opinion/wall-street-mispri...
4,6,2025-07-18-19-03-00 +0000,world,Brazil’s Bolsonaro Ordered to Wear Ankle Bracelet,https://www.wsj.com/world/americas/brazils-bol...
6,6,2025-07-18-14-55-00 +0000,world,EU Targets Russia With Toughest Sanctions Pack...,https://www.wsj.com/world/europe/eu-targets-ru...
5,5,2025-07-18-18-22-00 +0000,world,Opinion | Political Realism Requires Transatla...,https://www.wsj.com/opinion/embracing-americas...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
